In [1]:
import pybaseball
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
def swing_type(outcome_text):
    out_text = outcome_text.lower()
    swing_pattern = [r'field_out', r'single', r'double', r'triple',
                     r'home_run', r'grounded_into_double_play', r'force_out', 
                     r'sac_fly', r'field_error', r'fielders_choice', r'fielders_choice_out',
                     r'double_play', r'triple_play', r'swinging_strike', r'foul', r'foul_tip', r'swinging_strike_blocked']
    take_pattern = [r'ball', r'walk', r'hit_by_pitch', r'called_strike', r'called_strike', r'blocked_ball'] 

    for p in swing_pattern:
        if p in out_text:
            return 1
    for p in take_pattern:
        if p in out_text:
            return 0
    return None 



In [4]:
batter_info = pybaseball.statcast('2025-10-14', '2025-10-15')
batter_info

This is a large query, it may take a moment to complete


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
 50%|█████     | 1/2 [00:00<00:00,  2.20it/s]c:\Users\yjain\Desktop\Personal Work\moonshot\.venv\Lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
100%|██████████| 2/2 [00:00<00:00,  4.11it/s]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
165,FC,2025-10-15,90.3,3.66,5.66,"Fluharty, Mason",645302,689254,field_out,hit_into_play,...,5,2.06,-0.4,0.4,31.3,2.403379,30.452019,20.772955,40.169943,7.508847
174,FC,2025-10-15,91.6,3.67,5.67,"Fluharty, Mason",641598,689254,field_out,hit_into_play,...,<NA>,1.76,-0.09,0.09,33.2,16.710821,6.718283,30.394547,43.844998,29.77832
180,FC,2025-10-15,90.0,3.54,5.65,"Fluharty, Mason",641598,689254,NaN,ball,...,<NA>,1.95,-0.32,0.32,29.8,<NA>,<NA>,<NA>,<NA>,<NA>
189,ST,2025-10-15,81.1,3.53,5.77,"Fluharty, Mason",641598,689254,NaN,ball,...,<NA>,3.24,-1.2,1.2,35.6,<NA>,<NA>,<NA>,<NA>,<NA>
197,FC,2025-10-15,91.7,3.73,5.69,"Fluharty, Mason",553993,689254,strikeout,swinging_strike,...,1,1.54,-0.09,0.09,31.5,-3.049613,27.407045,42.523556,33.052072,18.263231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,SL,2025-10-14,83.9,-3.66,5.24,"Peralta, Freddy",660271,642547,strikeout,swinging_strike,...,2,3.05,-0.33,0.33,36.3,29.550436,-28.863768,45.627352,40.956874,42.490008
255,FF,2025-10-14,94.4,-3.69,5.18,"Peralta, Freddy",660271,642547,NaN,ball,...,2,1.31,0.65,-0.65,33.1,<NA>,<NA>,<NA>,<NA>,<NA>
262,CU,2025-10-14,76.7,-3.7,5.54,"Peralta, Freddy",660271,642547,NaN,ball,...,2,4.22,-0.57,0.57,35.8,<NA>,<NA>,<NA>,<NA>,<NA>
272,CU,2025-10-14,78.1,-3.59,5.56,"Peralta, Freddy",660271,642547,NaN,called_strike,...,2,4.21,-0.48,0.48,38.7,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
shortened_data = batter_info[['batter', 'pitch_type', 'description', 'plate_x', 'plate_z', 'events', 'release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'hc_x', 'hc_y', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'estimated_ba_using_speedangle', 'launch_speed_angle']]
pruned_data = shortened_data
print(pruned_data.shape)
for elem in ['batter', 'pitch_type', 'description', 'plate_x', 'plate_z', 'events', 'release_speed', 'release_pos_x', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'hc_x', 'hc_y', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'estimated_ba_using_speedangle', 'launch_speed_angle']:
    print(pruned_data[elem].head())

(558, 25)
165    645302
174    641598
180    641598
189    641598
197    553993
Name: batter, dtype: Int64
165    FC
174    FC
180    FC
189    ST
197    FC
Name: pitch_type, dtype: object
165      hit_into_play
174      hit_into_play
180               ball
189               ball
197    swinging_strike
Name: description, dtype: object
165   -0.03
174     0.5
180   -1.77
189    0.79
197    0.17
Name: plate_x, dtype: Float64
165     3.5
174    2.66
180    0.44
189    3.22
197    2.96
Name: plate_z, dtype: Float64
165    field_out
174    field_out
180          NaN
189          NaN
197    strikeout
Name: events, dtype: object
165    90.3
174    91.6
180    90.0
189    81.1
197    91.7
Name: release_speed, dtype: Float64
165    3.66
174    3.67
180    3.54
189    3.53
197    3.73
Name: release_pos_x, dtype: Float64
165    80.7
174    94.1
180    <NA>
189    <NA>
197    <NA>
Name: launch_speed, dtype: Float64
165       6
174      68
180    <NA>
189    <NA>
197    <NA>
Name: launch_angle, dty

In [10]:
both_na = pruned_data['hc_x'].isna() & pruned_data['hc_y'].isna()
both_na_2 = pruned_data['launch_speed'].isna() & pruned_data['launch_angle'].isna()
both_na_3 = pruned_data['launch_speed_angle'].isna() & pruned_data['estimated_ba_using_speedangle'].isna()

pruned_data['outcome_text'] = pruned_data['events']

pruned_data['outcome_text'] = pruned_data['description'].where(both_na, pruned_data['events'])


pruned_data.loc[both_na, 'hc_x'] = 0
pruned_data.loc[both_na, 'hc_y'] = 0
pruned_data.loc[both_na_2, 'launch_speed'] = 0
pruned_data.loc[both_na_2, 'launch_angle'] = 0
pruned_data.loc[both_na_3, 'launch_speed_angle'] = 0
pruned_data.loc[both_na_3, 'estimated_ba_using_speedangle'] = 0

pruned_data = pruned_data.dropna(subset=['outcome_text'])


C:\Users\yjain\AppData\Local\Temp\ipykernel_31320\1737895219.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruned_data['outcome_text'] = pruned_data['events']
C:\Users\yjain\AppData\Local\Temp\ipykernel_31320\1737895219.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pruned_data['outcome_text'] = pruned_data['description'].where(both_na, pruned_data['events'])


In [11]:
pruned_data['batting_pattern'] = pruned_data['outcome_text'].apply(swing_type)
pruned_data = pruned_data.dropna(subset=['batting_pattern'])
pruned_data['batting_pattern']

165    1.0
174    1.0
180    0.0
189    0.0
197    1.0
      ... 
249    1.0
255    0.0
262    0.0
272    0.0
279    0.0
Name: batting_pattern, Length: 557, dtype: float64

In [12]:
batter_enc = LabelEncoder() 
pruned_data['batter_id'] = batter_enc.fit_transform(pruned_data['batter'])
pitch_enc = LabelEncoder()
pruned_data['pitch_type_id'] = pitch_enc.fit_transform(pruned_data['pitch_type'])
outcome_enc = LabelEncoder()
pruned_data['outcome_id'] = outcome_enc.fit_transform(pruned_data['outcome_text'])
batter_pattern_enc = LabelEncoder()
pruned_data['batting_pattern_id'] = batter_pattern_enc.fit_transform(pruned_data['batting_pattern'])
launch_speed_angle_enc = LabelEncoder()
pruned_data['launch_speed_angle_id'] = launch_speed_angle_enc.fit_transform(pruned_data['launch_speed_angle'])

NUM_BATTERS = pruned_data['batter'].nunique()
NUM_PITCHES = pruned_data['pitch_type_id'].nunique()
NUM_OUTCOMES = pruned_data['outcome_id'].nunique()
NUM_BATTER_PATTERNS = pruned_data['batting_pattern_id'].nunique()
NUM_LAUNCH_SPEED_ANGLE = pruned_data['launch_speed_angle_id'].nunique()

scaler = StandardScaler()
pruned_data[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'hc_x', 'hc_y', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'estimated_ba_using_speedangle']] = scaler.fit_transform(pruned_data[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'hc_x', 'hc_y', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'estimated_ba_using_speedangle']].astype(float))
pruned_data.head()

print(pruned_data['outcome_text'].value_counts())

#pruned_data = pruned_data[pruned_data['events'] != 'catcher_interf']

print(pruned_data['outcome_id'].value_counts())
print(pruned_data['outcome_text'].values)
labels = set(pruned_data['outcome_text'].values)
labels = list(labels)
print(labels)
print(NUM_OUTCOMES)
print(NUM_PITCHES)
print(NUM_BATTER_PATTERNS)

outcome_text
ball                         165
called_strike                 97
foul                          92
swinging_strike               69
field_out                     65
single                        20
blocked_ball                  14
home_run                      11
double                         9
swinging_strike_blocked        3
grounded_into_double_play      3
force_out                      3
foul_tip                       3
hit_by_pitch                   2
field_error                    1
Name: count, dtype: int64
outcome_id
0     165
2      97
7      92
13     69
5      65
12     20
1      14
11     11
3       9
14      3
9       3
6       3
8       3
10      2
4       1
Name: count, dtype: int64
['field_out' 'field_out' 'ball' 'ball' 'swinging_strike' 'foul' 'ball'
 'foul' 'ball' 'foul' 'foul' 'foul' 'called_strike' 'ball' 'single'
 'field_out' 'called_strike' 'single' 'field_out' 'field_out' 'home_run'
 'called_strike' 'ball' 'called_strike' 'ball' 'ball' 'foul'
 'call

In [13]:
class SuperDataSet(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
    def __getitem__(self, index):
        row = self.df.loc[index]
        batter_id = int(row['batter_id'])
        pitch_type_id = int(row['pitch_type_id'])
        features = torch.tensor(row[['release_speed', 'release_pos_x', 'plate_x', 'plate_z', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'batting_pattern_id', 'launch_speed_angle_id', 'hc_x', 'hc_y', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'estimated_ba_using_speedangle']].values.astype(np.float32), dtype=torch.float32)
        y = int(row['outcome_id'])
        return batter_id, pitch_type_id, features, y
    def __len__(self):
        return len(self.df)




train_df, test_df = train_test_split(pruned_data, test_size=0.2, random_state=42)

train_dataset = SuperDataSet(train_df)
test_dataset = SuperDataSet(test_df)

print(f"Train dataset size: {len(train_dataset)}")
print(train_dataset[1])
print(f"Test dataset size: {len(test_dataset)}")
print(test_dataset[1])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)



Train dataset size: 445
(30, 2, tensor([-0.5738, -0.1241,  0.2362,  0.4379, -0.7091, -0.2246, -0.4631,  0.6489,
         0.2905,  1.0000,  0.0000, -0.4730, -0.4674,  0.0692,  0.5460,  0.3927,
         0.2598, -1.1206,  0.0955, -0.3634]), 13)
Test dataset size: 112
(5, 4, tensor([ 0.4632, -0.3389,  1.0046, -1.3807, -0.7091, -0.2246,  0.4651, -2.3813,
         0.2905,  1.0000,  0.0000, -0.4730, -0.4674,  0.9513, -0.4516, -0.7440,
        -0.8428,  0.5565, -0.4369, -0.3634]), 13)


In [15]:
class SuperModel(nn.Module):
    def __init__(self, num_batters, num_pitch_types, num_input_data_types, output_dim, batter_embedding=15, pitch_embedding=5, hidden_dim=64): #Mess around with batter_embedding, pitch_embedding, hidden_dim
        super().__init__()
        self.batter_embedding = nn.Embedding(num_batters, batter_embedding)
        self.pitch_embedding = nn.Embedding(num_pitch_types, pitch_embedding)
        self.linear1 = nn.Linear(num_input_data_types, 32)
        self.linear2 = nn.Linear(32, 16)

        self.fc1 = nn.Linear(batter_embedding + pitch_embedding + 16, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, batter_id, pitch_id, input_data):
        b = self.batter_embedding(batter_id)
        p = self.pitch_embedding(pitch_id) 
        i = self.linear1(input_data)
        i = self.relu(i)
        i = self.linear2(i)
        i = self.relu(i)
        x = torch.cat([b, p, i], dim=1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x # {'foul': 0.1, 'single': 0.3, ...}

In [16]:
model_t = SuperModel(num_batters=NUM_BATTERS, num_pitch_types=NUM_PITCHES, num_input_data_types=20, output_dim=NUM_OUTCOMES).to('cuda')
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_t.parameters(), lr=0.001)
print(model_t)


SuperModel(
  (batter_embedding): Embedding(41, 15)
  (pitch_embedding): Embedding(10, 5)
  (linear1): Linear(in_features=20, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=16, bias=True)
  (fc1): Linear(in_features=36, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=15, bias=True)
)


In [17]:
for epoch in range(20):
    model_t.train()
    for i, (batter_id, pitch_type_id, features, y) in enumerate(train_loader):
        batter_id = batter_id.to('cuda').long()
        pitch_type_id = pitch_type_id.to('cuda').long()
        features = features.to('cuda')
        y = y.to('cuda')
        logits = model_t(batter_id, pitch_type_id, features)
        loss_value = loss(logits, y)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss_value.item()}")



Epoch 1, Loss: 2.636997699737549
Epoch 1, Loss: 2.6838455200195312
Epoch 1, Loss: 2.770709753036499
Epoch 1, Loss: 2.488656997680664
Epoch 1, Loss: 2.5956459045410156
Epoch 1, Loss: 2.5748748779296875
Epoch 1, Loss: 2.7359135150909424
Epoch 1, Loss: 2.4772300720214844
Epoch 1, Loss: 2.513617515563965
Epoch 1, Loss: 2.5950262546539307
Epoch 1, Loss: 2.8401894569396973
Epoch 1, Loss: 2.3575518131256104
Epoch 1, Loss: 2.9299774169921875
Epoch 1, Loss: 2.632143497467041
Epoch 1, Loss: 2.3863487243652344
Epoch 1, Loss: 2.644646167755127
Epoch 1, Loss: 2.5596442222595215
Epoch 1, Loss: 2.5292258262634277
Epoch 1, Loss: 2.4348201751708984
Epoch 1, Loss: 2.2316813468933105
Epoch 1, Loss: 2.672398090362549
Epoch 1, Loss: 2.278134346008301
Epoch 1, Loss: 2.502051591873169
Epoch 1, Loss: 2.6802258491516113
Epoch 1, Loss: 2.663557529449463
Epoch 1, Loss: 2.5068254470825195
Epoch 1, Loss: 2.3741228580474854
Epoch 1, Loss: 2.358900547027588
Epoch 1, Loss: 2.45235013961792
Epoch 1, Loss: 2.4126029014

In [18]:
model_t.eval()
with torch.inference_mode():
    all = []
    probable = []
    for i, (batter_id, pitch_type_id, features, y) in enumerate(test_loader):
        batter_id = batter_id.to('cuda').long()
        pitch_type_id = pitch_type_id.to('cuda').long()
        features = features.to('cuda')
        y = y.to('cuda')
        logits = model_t(batter_id, pitch_type_id, features)
        #print(logits)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        #print(probs)

        for batch_idx, probs_row in enumerate(probs):
            pred_dict = {}
            max_value = 0
            outcome_probable = None
            for i in range(len(labels)):
                if float(probs_row[i]) > max_value:
                    max_value = probs_row[i]
                    outcome_probable = labels[i]
                pred_dict[labels[i]] = float(probs_row[i])
            
            all.append(pred_dict)
            probable.append((max_value,outcome_probable, batter_id[batch_idx], pitch_type_id[batch_idx]))
            print(pred_dict)
        loss_value = loss(logits, y)
        #print(f"Test Batch {i+1}, Loss: {loss_value.item()}")

{'double': 0.005714548751711845, 'field_out': 8.860311595526582e-08, 'home_run': 4.321794033756987e-09, 'force_out': 3.036035658987579e-11, 'single': 4.613277641840607e-12, 'foul_tip': 1.2406758287397679e-05, 'hit_by_pitch': 1.323971154931769e-09, 'grounded_into_double_play': 0.016442855820059776, 'swinging_strike_blocked': 5.130007048137486e-05, 'field_error': 5.948380898956884e-09, 'blocked_ball': 1.6105117538245395e-05, 'called_strike': 6.936990981643021e-10, 'foul': 4.001396121111611e-07, 'ball': 0.9777620434761047, 'swinging_strike': 2.1695294094570272e-07}
{'double': 0.0023506300058215857, 'field_out': 0.0002925870940089226, 'home_run': 1.14465674352221e-12, 'force_out': 4.4832846744953525e-13, 'single': 2.2836592710709543e-15, 'foul_tip': 3.7556242205027957e-06, 'hit_by_pitch': 2.5717647456069415e-10, 'grounded_into_double_play': 0.0009055575355887413, 'swinging_strike_blocked': 1.5400011761812493e-05, 'field_error': 1.3567266234082442e-11, 'blocked_ball': 1.4614725785122573e-07

In [19]:
probable

[(np.float32(0.97776204),
  'ball',
  tensor(2, device='cuda:0'),
  tensor(7, device='cuda:0')),
 (np.float32(0.9964169),
  'ball',
  tensor(5, device='cuda:0'),
  tensor(4, device='cuda:0')),
 (np.float32(0.9789853),
  'double',
  tensor(20, device='cuda:0'),
  tensor(7, device='cuda:0')),
 (np.float32(0.99912065),
  'grounded_into_double_play',
  tensor(14, device='cuda:0'),
  tensor(0, device='cuda:0')),
 (np.float32(0.9465146),
  'ball',
  tensor(1, device='cuda:0'),
  tensor(3, device='cuda:0')),
 (np.float32(0.5103658),
  'foul_tip',
  tensor(3, device='cuda:0'),
  tensor(6, device='cuda:0')),
 (np.float32(0.85731906),
  'ball',
  tensor(33, device='cuda:0'),
  tensor(3, device='cuda:0')),
 (np.float32(0.6348308),
  'ball',
  tensor(29, device='cuda:0'),
  tensor(9, device='cuda:0')),
 (np.float32(0.9999416),
  'foul_tip',
  tensor(8, device='cuda:0'),
  tensor(1, device='cuda:0')),
 (np.float32(0.9948258),
  'double',
  tensor(9, device='cuda:0'),
  tensor(7, device='cuda:0')),
